In [1]:
import nltk
from collections import defaultdict, Counter
import gensim, logging
import nltk
from gensim.models.keyedvectors import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from multiprocessing import Process, Pool
import os
import collections
from random import shuffle

In [2]:
# nltk.corpus.gutenberg.fileids()
# total_corpus = []
# for file in nltk.corpus.gutenberg.fileids():
#     with open((nltk.corpus.gutenberg.words(file).fileid.path), 'r') as f:
#         for line in f.readlines():
#             try:
#                 total_corpus.append(line.split())
#             except:
#                 print ("hi")
#                 print (line)

# with open(emma.fileid.path) as f:
#     for line in f.readlines():
#         print (line)

# emma = nltk.corpus.gutenberg.words('austen-emma.txt')
# vocab = list(set(emma))
# len(vocab)

In [3]:
def extract_patterns_in_words(patterns,pattern_counter,word1,word2,max_len):
    i = 1
    while(word1[:i] == word2[:i]):
        i = i + 1
    if i != 1 and i > max(len(word1[i-1:]), len(word2[i-1:])) < max_len:
        pattern_counter[("suffix",word1[i-1:], word2[i-1:])] += 1
        if ("suffix",word1[i-1:], word2[i-1:]) in patterns:
            patterns[("suffix",word1[i-1:], word2[i-1:])].append((word1, word2))
        else:
            patterns[("suffix",word1[i-1:], word2[i-1:])] = [(word1, word2)]
#         patterns[("suffix",word1[i-1:], word2[i-1:], word1, word2)] += 1
    i = 1
    while(word1[-i:] == word2[-i:]):
        i = i + 1
    if i != 1 and max(len(word1[:-i+1]), len(word2[:-i+1])) < max_len:
        pattern_counter[("prefix",word1[:-i+1], word2[:-i+1])] += 1
        if ("prefix",word1[:-i+1], word2[:-i+1]) in patterns:
            patterns[("prefix",word1[:-i+1], word2[:-i+1])].append((word1, word2))
        else:
            patterns[("prefix",word1[:-i+1], word2[:-i+1])] = [(word1, word2)]
#         patterns[("prefix",word1[:-i+1], word2[:-i+1], word1, word2)] += 1
    return patterns

In [10]:
def build_pattern_dict(vocab,max_len = 6):
    patterns  = defaultdict(list)
    print (patterns)
    pattern_counter = Counter()
    for word in vocab:
        for second_word in vocab:
            if word != second_word:
                extract_patterns_in_words(patterns,pattern_counter,word,second_word,max_len)
    return patterns, pattern_counter

In [14]:
def downsample_patterns():
    #Downsample to include only top 1000
    pattern_1000 = defaultdict(list)
    for pattern,items in patterns.items():
        shuffle(items)
        pattern_1000[pattern] = items[:1000]
    return pattern_1000

In [16]:
def pair_wise_similarity(word_pair1, word_pair2,topn = 10):
    closest_n = word_vectors.most_similar(positive=[word_pair1[0], word_pair2[0]], negative=[word_pair2[1]], topn=topn)
#     print (word_pair1, word_pair2)
#     print (closest_n)
    for word, cos_sim in closest_n:
        if word == word_pair2[1]:
            return True
    return False
    

In [20]:
def index_vector(word_vectors, dimensions=300):
    fname = '../data/annoy.index'
    # Persist index to disk
    if os.path.exists(fname):
        annoy_index = AnnoyIndexer()
        annoy_index.load(fname)
        annoy_index.model = word_vectors
    else:
        annoy_index = AnnoyIndexer(word_vectors, dimensions)
    annoy_index.save(fname)
    return annoy_index

In [21]:
def annoy_pair_wise_similarity(word_pair1, word_pair2,topn = 10):
    closest_n = word_vectors.most_similar(positive=[word_pair1[0], word_pair2[0]], negative=[word_pair2[1]], topn=topn, indexer=annoy_index)
#     print (word_pair1, word_pair2)
#     print (closest_n)
    for word, cos_sim in closest_n:
        if word == word_pair2[1]:
            return True
    return False
    

In [47]:
def calculate_hit_rate(patterns, word_vectors, similarity_function):
    hit_rates = {}
    for (pattern,support_set) in patterns.items():
        hit_count = 0
        for pair1 in support_set:
            for pair2 in support_set:
                if pair1 != pair2 and similarity_function(pair1, pair2, 20):
                    hit_count += 1
        hit_rates[pattern] = hit_count / float(len(support_set))
    return hit_rates

## Most Common patterns

## Least Common Patterns

In [5]:
#pattern_counter.most_common()[:-20:-1]

In [32]:
%time word_vectors = KeyedVectors.load_word2vec_format('/home/raja/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=5000)

CPU times: user 260 ms, sys: 0 ns, total: 260 ms
Wall time: 259 ms


In [27]:
%time patterns, pattern_counter = build_pattern_dict(word_vectors.vocab.keys())

defaultdict(<class 'list'>, {})
CPU times: user 21.8 s, sys: 20 ms, total: 21.8 s
Wall time: 21.8 s


In [28]:
pattern_counter.most_common(20)

[(('suffix', 's', ''), 598),
 (('suffix', '', 's'), 598),
 (('suffix', 'ed', ''), 168),
 (('suffix', '', 'ed'), 168),
 (('suffix', '', 'ing'), 149),
 (('suffix', 'ing', ''), 149),
 (('suffix', 'ing', 'ed'), 141),
 (('suffix', 'ed', 'ing'), 141),
 (('suffix', '', 'd'), 127),
 (('suffix', 'd', ''), 127),
 (('suffix', 'ed', 's'), 86),
 (('suffix', 's', 'ed'), 86),
 (('suffix', 'ly', ''), 80),
 (('suffix', '', 'ly'), 80),
 (('suffix', 's', 'ing'), 76),
 (('suffix', 'ing', 's'), 76),
 (('suffix', 'ing', 'e'), 74),
 (('suffix', 'e', 'ing'), 74),
 (('suffix', 's', 'd'), 62),
 (('suffix', 'd', 's'), 62)]

In [31]:
%time annoy_index = index_vector(word_vectors=word_vectors, dimensions=300)

CPU times: user 1.35 s, sys: 308 ms, total: 1.66 s
Wall time: 1.83 s


In [35]:
%time sampled_patterns = downsample_patterns()

CPU times: user 3.36 s, sys: 0 ns, total: 3.36 s
Wall time: 3.36 s


In [ ]:
%time hit_rates = calculate_hit_rate(sampled_patterns, word_vectors, pair_wise_similarity)

In [ ]:
type(hit_rates)

In [ ]:
co = Counter(hit_rates)

In [ ]:
co.most_common(20)

In [ ]:
sampled_patterns[('prefix', 'hai', 'close')]